In [1]:
# Loading data and 
import pickle
import plotly.express as px
import pandas as pd
import numpy as np
import torch
from LogLikelihoodTorch import log_likelihood,Optimizationloop, Initialize,log_likelihood_eval,OptimizationTraj
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f)
torch.set_default_dtype(torch.float64)
X_tensor = torch.from_numpy(X).to(torch.float64)

In [ ]:
# Setting up plot standards
import plotly.express as px
import plotly.io as pio

pio.templates.default = "simple_white"

px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
px.defaults.color_discrete_sequence = px.colors.qualitative.G10
px.defaults.width = 800
px.defaults.height = 600


In [ ]:
# Estimating Likelihood trajectory for each learning rate
n_iters = 3000
LearningRates = [0.1,0.5,0.8,1,1.2,1.5]
Likelihoods = np.zeros((len(LearningRates),n_iters))

for idx,lr in enumerate(LearningRates):
    print(lr)
    pi,kappa,mu = Initialize(p=p,K=K)
    Parameters = [
        {'params':pi},
        {'params':kappa},
        {'params':mu}
    ]
    Adam = torch.optim.Adam(Parameters,lr=lr)
    Likelihoods[idx,:] = OptimizationTraj(X_tensor,[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

In [ ]:
# Setup Dataframe for plotting
TrajectoryData = pd.DataFrame()
for idx,lr in enumerate(LearningRates):
    TrajectoryData[f'lr= {lr}'] = Likelihoods[idx,:]

fig=px.line(TrajectoryData,
color_discrete_sequence=px.colors.qualitative.G10
)
fig.update_layout(
    title="Learning Rate Evaluations",
    width=800,height=600,
    legend={'title':"Learning Rates"}
    #template="plotly_white",
)
fig.update_yaxes(
    title="log-Likelihood"
)
fig.update_xaxes(
    title="Epoch"
)

fig.show()